### Declaración de rutas y acceso a otro notebooks

In [1]:
%run APYME_bd.ipynb
%run APYME_Modelo.ipynb

path = '/Users/mb66483/Downloads/Crédito PyME/'


### Lectura de bases de datos

In [2]:
df_universo, df_sunat, df_sunart, df_rcc, df_ranking, df_digital, df_transac = read_bd(path)


db_ranking = clean_db_ranking(df_ranking)


df_sunart_1 = transf_sunart()
df_rcc_5 = transf_rcc()
df_digital = transf_digital(df_digital)
df_transac_6 = transf_transac(df_transac)


In [15]:
df_join, df_features = join_tables()
df_join = df_join[(df_join.income_monthly_amount>0)]

#### Eliminar variables correlaciondas y no significativas

In [16]:
vo = 'vo_balance_amount'
varCorr = variablesCorrelacionadas(df_join,df_features,vo)
df_features2 = list(set(df_features)-set(varCorr))

#
df_features = varOrdenporCorr(df_join,df_features2,vo, True)

In [21]:
df_features

['traspaso_ctas_ct',
 'traspaso_entre_ctes_ct',
 'employees_number',
 'pbi_sector_last_per',
 'ranking_last_number',
 'income_monthly_amount',
 'branches_number',
 'min_pen_billing_last_amount',
 'traspaso_entre_ctes_sm',
 'veh_ct',
 'veh_year',
 'birth_date',
 'trasf_env_ct',
 'market_share_per',
 'trasf_env_sm',
 'u_online_banking_affiliation_type',
 'u_digital_affiliation_type',
 'u_sms_affiliation_type',
 'dias_digital',
 'trasf_rec_ct',
 'trasf_rec_sm',
 'dias_online',
 'dias_sms']

### Modelos supervisados

In [18]:
df = df_join
vo = 'vo_balance_amount'

X_train, X_test, y_train, y_test = partition(df,df_features,vo)
y_train =pd.DataFrame(y_train,columns=['vo_balance_amount'])

#### SVM

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

In [ ]:
y_predict = clf.predict(X_test)
y_predict

#### Random Forest

In [134]:

from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=0)
regr.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [135]:
y_predict = regr.predict(X_test)
y_predict

array([ 425155.2761, 1304831.0451,  552691.0407, ...,  197315.3627,
        738176.6794,  452561.2184])

#### Arbol de decisión

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeRegressor(random_state=0)
clf = clf.fit(X_train, y_train)

In [ ]:
y_predict = clf.predict(X_test)
y_predict

#### Stochastic gradient descent 

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=50, random_state=3)
clf.fit(X_train, y_train)


In [ ]:
y_predict = clf.predict(X_test)
y_predict

#### Regresión Ridge

In [ ]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.5, random_state=1, normalize=True)
reg.fit(X_train, y_train)


In [ ]:
y_predict = reg.predict(X_test)
y_predict

#### Regresión lineal OLS

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
y_predict = reg.predict(X_test)
y_predict

#### Regresión Lasso

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1, normalize=True)
clf.fit(X_train, y_train)

y_predict = clf.predict(X_test)
y_predict


#### Red neuronal 
##### Se hace la optimización con los hiperparámetros

In [19]:
import pickle
ruta = '/Users/mb66483/Documents/Modelo PYME/pkl/'
#dfres2T, dfres3T = pd.DataFrame(), pd.DataFrame()
dfres2A, dfres3A = pd.DataFrame(), pd.DataFrame()
error=100000000
transformacion = ['minmax','standard','robust']
regrR=''
for transf in transformacion:
    print('//TRANSFORMACION: '+transf)
#print('BANCO: '+banc+'//LAGS: '+ str(lags)+'//VARIABLE: '+column+'//TRANSFORMACION: '+transf)
    X_train2, X_test2, y_train2, mm_scaler, mm_scaler2 = ftransformacion(X_train, X_test, y_train, transf)
    X_train2 =X_train
    X_test2= X_test
    y_train2 = y_train
    
    activation = ['identity','relu']
    for act in activation:
        for i in range(0,10):

            y_predict2, regr = model_nn(X_train2, X_test2, y_train2.values.ravel(), i,act)
            y_predict = transformacion_inversa(y_predict2, mm_scaler2)
            dfcom2, dfcom3 =  resultados(df)

            error_i = dfcom3
            #print(error_i[0])
            if error_i<error:

                dfres2A = dfcom2
                dfres3A = dfcom3   
                
                regrR = regr
                
                print("    activation: "+act+" i: "+str(i)+' error: '+ str(error) + " error_i: "+str(error_i))
                error = dfcom3   
pickle.dump(regrR, open(ruta+'PYMEmodel.sav', 'wb'))
      
dfres2A.to_excel('/Users/mb66483/Documents/Modelo PYME/resultadosModeloPyme.xlsx')

//TRANSFORMACION: minmax
    activation: identity i: 0 error: 100000000 error_i: 154820.7676709406
    activation: identity i: 1 error: 154820.7676709406 error_i: 154082.36121092056
    activation: identity i: 2 error: 154082.36121092056 error_i: 151873.15003424673
    activation: identity i: 9 error: 151873.15003424673 error_i: 150716.7349143327
    activation: relu i: 1 error: 150716.7349143327 error_i: 148570.25192435874
    activation: relu i: 2 error: 148570.25192435874 error_i: 148275.73230527542
    activation: relu i: 3 error: 148275.73230527542 error_i: 148125.34117934594
    activation: relu i: 7 error: 148125.34117934594 error_i: 144493.66135601132
//TRANSFORMACION: standard
//TRANSFORMACION: robust


In [28]:
df.to_csv('/Users/mb66483/Documents/baseTrain.csv')

In [20]:
dfres2A

,index,personal_id_x,customer_id,sbs_customer_id,taxpayer_id,segmento,gender_type,vo_balance_amount,valor_pred,error_est,p_error_est
0,1,39025845,44955658,8268872505,11854064724,PNN,M,292781.89,123461.545628,169320.344372,0.578316
1,3,83084480,67646864,5908385493,20489054918,PNN,F,29615.68,107829.986638,-78214.306638,2.640976
2,4,49400873,18301674,3731068026,72277039581,PNN,F,659085.27,113815.442982,545269.827018,0.827313
3,5,49536356,95253809,3278182181,81323002183,PNN,F,100000.00,153839.480129,-53839.480129,0.538395
4,6,04879638,98052241,0499167952,51808861075,PNN,M,42000.00,91020.498783,-49020.498783,1.167155
...,...,...,...,...,...,...,...,...,...,...,...
2095,26965,24770931,48153143,7842011123,80007916156,PNN,M,20004.20,58761.853851,-38757.653851,1.937476
2096,26967,92755754,28388932,1880394102,95509014820,PNN,F,30465.00,61676.266967,-31211.266967,1.024496
2097,26968,88713336,88794803,2396530322,17081841326,PNN,M,118400.00,42105.965969,76294.034031,0.644375
2098,26971,61984330,80228448,4227844091,39612419378,PNN,M,15000.00,155889.910010,-140889.910010,9.392661


### Modelos no supervisados

In [ ]:
df_universo, df_sunat, df_sunart, df_rcc, df_ranking, df_digital, df_transac = read_bd(path)


db_ranking = clean_db_ranking(df_ranking)
db_ranking.sample(5)

df_sunart_1 = transf_sunart()
df_rcc_5 = transf_rcc()
df_digital = transf_digital(df_digital)
df_transac_6 = transf_transac(df_transac)

In [207]:
df_sunart_1 = transf_sunart()

#### Sunart

In [212]:
from sklearn.cluster import KMeans

sunart_columns = ['veh_ct','veh_year']
df_sunart_1.update(df_sunart_1[sunart_columns].fillna(0))



kmeans = KMeans(n_clusters=10, random_state=0).fit(df_sunart_1[sunart_columns])
y_pred = kmeans.predict(df_sunart_1[sunart_columns])

#####
df_y_pred = pd.DataFrame(y_pred, columns=['cluster'])

In [214]:
df_sunart_1 = df_sunart_1.reset_index()
df_kmeans_result = pd.concat([df_sunart_1,df_y_pred],axis=1)
#
df_kmeans_result_1 = df_kmeans_result.groupby('cluster').agg(ct=('index','count'), 
                                                             num_veh=('veh_ct','mean'), year_veh=('veh_year','mean')).reset_index()
df_kmeans_result_2 = df_kmeans_result_1.sort_values('num_veh',ascending=False)
df_kmeans_result_2

,cluster,ct,num_veh,year_veh
8,8,28,29.714286,2010.892857
5,5,267,8.962547,2012.067416
4,4,3515,1.691892,2012.667425
0,0,2147,1.613414,2008.600838
9,9,2,1.500000,86.500000
7,7,1305,1.446743,2003.281226
3,3,1955,1.438363,1995.310486
2,2,834,1.327338,1985.631894
6,6,311,1.151125,1972.604502
1,1,1051,1.000951,0.000000


#### Digital

In [216]:

digital_columns = ['u_digital_affiliation_type','u_online_banking_affiliation_type','u_sms_affiliation_type',
                   'dias_digital','dias_online','dias_sms']
df_digital.update(df_digital[digital_columns].fillna(0))



kmeans = KMeans(n_clusters=10, random_state=0).fit(df_digital[digital_columns])
y_pred = kmeans.predict(df_digital[digital_columns])

#####
df_y_pred = pd.DataFrame(y_pred, columns=['cluster'])

In [223]:
df_digital_1 = df_digital.reset_index()
df_kmeans_result = pd.concat([df_digital_1,df_y_pred],axis=1)
#
df_kmeans_result_1 = df_kmeans_result.groupby('cluster').agg(ct=('index','count'), 
                                                             num_digital=('u_digital_affiliation_type','sum'), dias_digital=('dias_digital','mean'),\
                                                            num_online=('u_online_banking_affiliation_type','sum'), dias_online=('dias_online','mean')
                                                            ).reset_index()
df_kmeans_result_2 = df_kmeans_result_1.sort_values('num_digital',ascending=False)
df_kmeans_result_2


,cluster,ct,num_digital,dias_digital,num_online,dias_online
3,3,4187,4187,490.619298,4187,540.310485
7,7,3452,3452,210.829664,3452,226.107764
5,5,2718,2718,996.407285,2718,1161.601177
1,1,772,772,1379.831606,772,1447.067358
0,0,5194,0,43887.000000,5194,1441.329226
2,2,3260,0,43887.000000,0,43887.000000
4,4,2048,0,43887.000000,2048,2711.990234
6,6,3135,0,43887.000000,3135,490.779266
8,8,3023,0,43887.000000,3023,2076.625868
9,9,301,0,43887.000000,301,4164.196013


In [ ]:
df_transac_6 = transf_transac(df_transac)

#### Transferencias

In [224]:
from sklearn.cluster import KMeans

transac_columns= ['trasf_env_ct', 'trasf_env_sm', 'trasf_rec_ct',
           'trasf_rec_sm', 'traspaso_ctas_ct', 'traspaso_ctas_sm',
           'traspaso_entre_ctes_ct', 'traspaso_entre_ctes_sm']
df_transac_6.update(df_transac_6[transac_columns].fillna(0))



kmeans = KMeans(n_clusters=10, random_state=0).fit(df_transac_6[transac_columns])
y_pred = kmeans.predict(df_transac_6[transac_columns])

#####
df_y_pred = pd.DataFrame(y_pred, columns=['cluster'])

In [225]:
df_transac_7 = df_transac_6.reset_index()
df_kmeans_result = pd.concat([df_transac_7,df_y_pred],axis=1)
#
df_kmeans_result_1 = df_kmeans_result.groupby('cluster').agg(ct=('index','count'), 
                                                             traspaso_entre_ctes_sm=('traspaso_entre_ctes_sm','mean'), traspaso_ctas_sm=('traspaso_ctas_sm','mean')).reset_index()
df_kmeans_result_2 = df_kmeans_result_1.sort_values('traspaso_entre_ctes_sm',ascending=False)
df_kmeans_result_2

,cluster,ct,traspaso_entre_ctes_sm,traspaso_ctas_sm
4,4,2,6.566603e+08,0.000000e+00
3,3,1,4.896468e+08,2.544573e+09
7,7,5,2.167834e+08,1.187200e+05
6,6,1,1.201452e+08,2.914275e+08
5,5,55,6.633151e+07,9.430125e+04
2,2,1,1.477319e+07,0.000000e+00
9,9,20,1.365109e+07,1.759864e+07
8,8,496,1.266606e+07,6.391838e+05
0,0,671386,4.217592e+04,2.514447e+03
1,1,1,5.500000e+03,0.000000e+00


#### Transferencias
##### Terceras partes vinculadas

In [3]:
df_vinc = df_transac.groupby(['COD_PERSONOR','COD_PERSONDE']).agg(sm=('IMP_IMPOPER','sum')).reset_index()
df_vinc_1 = df_vinc[df_vinc.COD_PERSONOR!=df_vinc.COD_PERSONDE]

In [4]:
df_vinc_2 = df_vinc_1.merge(df_vinc_1, how='left', on='COD_PERSONDE')
df_vinc_2 = df_vinc_2[df_vinc_2.COD_PERSONOR_x!=df_vinc_2.COD_PERSONOR_y]


In [5]:
df_vinc_3 = df_vinc_2.groupby(['COD_PERSONOR_x','sm_x']).agg(sm=('sm_y','sum')).reset_index()
df_vinc_3['monto_propio_vinc'] = df_vinc_3.sm_x + df_vinc_3.sm

In [6]:
df_vinc_4 = df_vinc_3.groupby('COD_PERSONOR_x').agg(monto_propio=('sm_x','sum'), monto_propio_vinc=('monto_propio_vinc','sum')).reset_index()

df_vinc_4['rnk_monto_propio'] = df_vinc_4.monto_propio.rank(pct=True, method='max')
df_vinc_4['rnk_monto_propio_vinc'] = df_vinc_4.monto_propio_vinc.rank(pct=True, method='max')




In [7]:
def cat(x):
    if x < 0.1: return 1
    elif x <0.2: return 2
    elif x <0.3: return 3
    elif x <0.4: return 4
    elif x <0.5: return 5
    elif x <0.6: return 6
    elif x <0.7: return 7
    elif x <0.8: return 8
    elif x <0.9: return 9
    else: return 10

df_vinc_4['cod_monto_propio'] = df_vinc_4['rnk_monto_propio'].map(cat)
df_vinc_4['cod_monto_propio_vinc'] = df_vinc_4['rnk_monto_propio_vinc'].map(cat)


In [21]:

df0 = df_vinc_4.groupby('cod_monto_propio').agg(mi=('monto_propio','min'), ma=('monto_propio','max')).reset_index()
df0['intervalo'] = "["+df0.mi.astype(str) + " - "+df0.ma.astype(str)+"]"

df1 = df_vinc_4.groupby('cod_monto_propio_vinc').agg(mi=('monto_propio_vinc','min'), ma=('monto_propio_vinc','max')).reset_index()
df1['intervalo'] = "["+df1.mi.astype(str) + " - "+df1.ma.astype(str)+"]"

In [28]:
df_vinc_5 = df_vinc_4.merge(df0[['cod_monto_propio','intervalo']],how='left', on='cod_monto_propio')\
                        .merge(df1[['cod_monto_propio_vinc','intervalo']],how='left', on='cod_monto_propio_vinc')\

In [30]:
# Matriz que muestra el número de clientes en deciles, los cuales cambiaron por la agregación de los montos vinculados
pd.crosstab([df_vinc_5.cod_monto_propio,df_vinc_5.intervalo_x], [df_vinc_5.cod_monto_propio_vinc,df_vinc_5.intervalo_y], values=df_vinc_5.COD_PERSONOR_x, 
            aggfunc='count').fillna("")



,cod_monto_propio_vinc,1,2,3,4,5,6,7,8,9,10
,intervalo_y,[1.01 - 37268.55],[37269.5 - 159980.82],[159995.94 - 435535.41000000003],[435651.48 - 986462.69],[986604.8499999999 - 1841068.79],[1841151.8199999998 - 3225838.74],[3227256.7800000003 - 5726149.689999996],[5726596.209999999 - 10837124.84000001],[10839113.609999994 - 23378837.140000004],[23380508.330000002 - 1406962277.7699986]
cod_monto_propio,intervalo_x,,,,,,,,,,
1,[0.01 - 1805.0],2049,820,512,282.0,209.0,126.0,81.0,39.0,22.0,16.0
2,[1806.84 - 6235.0],1158,860,710,489.0,361.0,247.0,140.0,92.0,69.0,31.0
3,[6238.7 - 14707.95],633,794,664,594.0,493.0,391.0,262.0,157.0,123.0,46.0
4,[14713.54 - 28930.6],272,677,640,665.0,605.0,487.0,327.0,229.0,173.0,82.0
5,[28935.0 - 52710.0],44,571,590,591.0,581.0,561.0,459.0,353.0,256.0,151.0
6,[52728.62 - 92605.0],,336,459,560.0,584.0,575.0,582.0,470.0,355.0,236.0
7,[92606.89 - 167986.27000000005],,99,385,455.0,521.0,609.0,643.0,617.0,472.0,356.0
8,[167995.02 - 325011.83],,,184,339.0,431.0,587.0,682.0,761.0,638.0,535.0


In [67]:
df_vinc_4[df_vinc_4.monto_propio>df_vinc_4.monto_propio_vinc]

,COD_PERSONOR_x,monto_propio,monto_propio_vinc,rnk_monto_propio,rnk_monto_propio_vinc,cod_monto_propio,cod_monto_propio_vinc
